In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, Dataset

import spacy
import numpy as np
import random
import math
import time
from tqdm import tqdm
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

In [2]:
#SEED = 1234

#random.seed(SEED)
#np.random.seed(SEED)
#torch.manual_seed(SEED)
#torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [5]:
# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

In [6]:
# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# normalize unicode characters
			line = normalize('NFD', line).encode('ascii', 'ignore')
			line = line.decode('UTF-8')
			# tokenize on white space
			line = line.split()
			# convert to lowercase
			#line = [word.lower() for word in line]
			# remove punctuation from each token
			line = [word.translate(table) for word in line]
			# remove non-printable chars form each token
			line = [re_print.sub('', w) for w in line]
			# remove tokens with numbers in them
			line = [word for word in line if word.isalpha()]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

In [7]:
# load dataset
filename = '../data/deu.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
clean_pairs = clean_pairs[0:12000]

In [8]:
np.random.shuffle(clean_pairs)
train_clean = clean_pairs[0:10000]
val_clean = clean_pairs[10000:11000]
test_clean = clean_pairs[11000:]
print(val_clean.shape)
print(val_clean[0:10])
print(train_clean.shape)
print(train_clean[0:10])
print(test_clean.shape)
print(test_clean[0:10])


(1000, 2)
[['Take Tom home' 'Bring Tom nach Hause']
 ['She misses him' 'Er fehlt ihr']
 ['Well cooperate' 'Wir werden kooperieren']
 ['Try harder' 'Strengen Sie sich mehr an']
 ['Whose is this' 'Wem gehort das']
 ['Theyre adorable' 'Sie sind sehr su']
 ['Were fasting' 'Wir fasten']
 ['I wont help you' 'Ich werde euch nicht helfen']
 ['Its so simple' 'Es ist so einfach']
 ['Youre in love' 'Du bist verliebt']]
(10000, 2)
[['Youre kind' 'Du bist nett']
 ['Tom is no expert' 'Tom ist nicht vom Fach']
 ['I was jealous' 'Ich war eifersuchtig']
 ['Tom was impolite' 'Tom war unhoflich']
 ['A dog is barking' 'Ein Hund bellt']
 ['Tom was angry' 'Tom war wutend']
 ['I am undressing' 'Ich entkleide mich']
 ['Watch Tom' 'Sieh Tom zu']
 ['Just a moment' 'Einen Augenblick']
 ['Whats it worth' 'Was ist es denn wert']]
(1000, 2)
[['Were busy' 'Wir sind beschaftigt']
 ['Do you have them' 'Hast du sie']
 ['Tom slipped' 'Tom rutschte aus']
 ['Tom needs a bath' 'Tom braucht ein Bad']
 ['Who panicked' 'Wer b

In [9]:
eng_file = "./.data/multi30k/train.en"
ge_file = "./.data/multi30k/train.de"
en_pt = open(eng_file,"w")
ge_pt = open(ge_file,"w")

for p in train_clean:
    en_pt.write(p[0]+"\n")
    ge_pt.write(p[1]+"\n")
    
en_pt.close()
ge_pt.close()

eng_file = "./.data/multi30k/val.en"
ge_file = "./.data/multi30k/val.de"
en_pt = open(eng_file,"w")
ge_pt = open(ge_file,"w")

for p in val_clean:
    en_pt.write(p[0]+"\n")
    ge_pt.write(p[1]+"\n")
    
en_pt.close()
ge_pt.close()

eng_file = "./.data/multi30k/test2016.en"
ge_file = "./.data/multi30k/test2016.de"
en_pt = open(eng_file,"w")
ge_pt = open(ge_file,"w")

for p in test_clean:
    en_pt.write(p[0]+"\n")
    ge_pt.write(p[1]+"\n")
    
en_pt.close()
ge_pt.close()

In [10]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    #print(text)
    #text = text.translate(str.maketrans('', '', string.punctuation))
    #print(text)
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [11]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    #print(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    #print(text)
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [12]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

C:\Users\MOHIT\Anaconda3\envs\torch\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [13]:
train_data,valid_data,test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

C:\Users\MOHIT\Anaconda3\envs\torch\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [14]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 10000
Number of validation examples: 1000
Number of testing examples: 1000


In [15]:
print(vars(train_data.examples[39]))

{'src': ['druck', 'tom'], 'trg': ['hug', 'tom']}


In [16]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [17]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 2043
Unique tokens in target (en) vocabulary: 1578


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

C:\Users\MOHIT\Anaconda3\envs\torch\lib\site-packages\torchtext\data\iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        #print("src2",src.shape)
        embedded = self.dropout(self.embedding(src))
        #print("embed",embedded.shape)
        #embedded = [sentence len, batch size, emb dim]
        #outputs (the top-layer hidden state for each time-step), hidden (the final hidden state for each layer, 
        #h_T, stacked on top of each other) and cell (the final cell state for each layer, c_T, 
        #stacked on top of each other).
        #n_directions=1 because the LSTM is not bidirectional
        outputs, (hidden, cell) = self.rnn(embedded)
        #print("output",outputs.shape)
        #print("hidden",hidden.shape)
        #print("cell",cell.shape)
        
        #outputs = [sentence len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        
        return hidden, cell

In [21]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        #self.softmax = nn.LogSoftmax(dim=0)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        #print(prediction.shape)
        #prediction = self.softmax(prediction)
        #print(prediction.shape)
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            #print(top1.shape)
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [23]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [24]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2043, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1578, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=1578, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,092,906 trainable parameters


In [26]:
optimizer = optim.Adam(model.parameters())

In [27]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [28]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(tqdm(iterator)):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        

    return epoch_loss / len(iterator)

In [29]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(tqdm(iterator)):

            src = batch.src
            #print("src1",src.shape)
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            #print("output b4",output.shape)
            #print(output[0,0,0])
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #print("output",output.shape)
            #print("trg",trg.shape)
            
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [33]:
def evaluate_sentence(sentence):
    sentence = sentence.lower()
    tokens = tokenize_de(sentence)
    #print(tokens)
    tokens.insert(0, SRC.init_token)
    tokens.append(SRC.eos_token)
    #print(tokens)
    text_to_indices = [SRC.vocab.stoi[token] for token in tokens]
    #print(text_to_indices)
    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)
    #print(sentence_tensor)
    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)
        
    outputs = [TRG.vocab.stoi["<sos>"]]

    for _ in range(50):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()
            #print(best_guess)

        outputs.append(best_guess)
        
        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == TRG.vocab.stoi["<eos>"]:
            break

    translated_sentence = [TRG.vocab.itos[idx] for idx in outputs]
    print(translated_sentence[1:])
    

In [34]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [36]:
evaluate_sentence("Wir haben eine gefunden".lower())

['eating', 'eating', 'grow', 'eating', 'eating', 'hurting', 'tried', 'eating', 'eating', 'eating', 'eating', 'insane', 'insane', 'eating', 'grow', 'eating', 'eating', 'cds', 'cds', 'cds', 'cds', 'cds', 'grow', 'eating', 'whose', 'whose', 'whose', 'farmer', 'eating', 'tried', 'eating', 'eating', 'whose', 'whose', 'whose', 'eating', 'tried', 'lazy', 'eating', 'eating', 'grow', 'eating', 'eating', 'eating', 'eating', 'tried', 'cds', 'whose', 'eating', 'eating']


In [46]:
model.load_state_dict(torch.load('../models/model-v2.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 44.59it/s]

| Test Loss: 2.233 | Test PPL:   9.323 |


In [47]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    evaluate_sentence("Wir haben eine gefunden")
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '../models/model-v2.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

  0%|                                                                                          | 0/157 [00:00<?, ?it/s]

['we', 'found', 'one', '<eos>']
Epoch: 01 | Time: 0m 5s
	Train Loss: 1.390 | Train PPL:   4.015
	 Val. Loss: 2.204 |  Val. PPL:   9.066


  0%|                                                                                          | 0/157 [00:00<?, ?it/s]

['we', 'found', 'one', '<eos>']
Epoch: 02 | Time: 0m 5s
	Train Loss: 1.261 | Train PPL:   3.529
	 Val. Loss: 2.183 |  Val. PPL:   8.874


  0%|                                                                                          | 0/157 [00:00<?, ?it/s]

...

['we', 'found', 'one', '<eos>']
Epoch: 13 | Time: 0m 32s
	Train Loss: 0.474 | Train PPL:   1.606
	 Val. Loss: 2.092 |  Val. PPL:   8.102


  0%|                                                                                          | 0/157 [00:00<?, ?it/s]

['we', 'found', 'one', '<eos>']
Epoch: 14 | Time: 0m 32s
	Train Loss: 0.437 | Train PPL:   1.548
	 Val. Loss: 2.094 |  Val. PPL:   8.118


 31%|█████████████████████████▎                                                       | 49/157 [00:10<00:22,  4.90it/s]


KeyboardInterrupt: 

In [50]:
model.load_state_dict(torch.load('../models/model-v2.pt'))

test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

#random.choice(test_clean)
i=0
while i<10:
    test_pair = random.choice(test_clean)
    ground_truth = test_pair[0].lower()
    to_test = test_pair[1]
    print(to_test)
    evaluate_sentence(to_test)
    print(ground_truth)
    i+=1

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 21.92it/s]


| Test Loss: 2.061 | Test PPL:   7.854 |
Seid ihr neugierig
['are', 'you', 'students', '<eos>']
are you curious
Ich liebe Witze
['i', 'love', 'you', '<eos>']
i love jokes
War das ein Nein
['was', 'it', 'a', 'joke', '<eos>']
was that a no
Ich lerne Koreanisch
['i', 'm', '<unk>', '<eos>']
i study korean
Probiere es einfach mal aus
['try', 'it', 'once', 'more', '<eos>']
just try it out
Du gefallst mir
['you', 've', 'me', '<eos>']
i like you
Recyceln Sie
['<unk>', 'you', '<eos>']
do you recycle
Es ist ein Gerucht
['its', 'a', 'ambush', '<eos>']
its hearsay
Ich bitte Sie lassen Sie mich ein
['ill', 'you', '<eos>']
please let me in
Ich bin hungrig
['i', 'am', 'hungry', '<eos>']
i am hungry
